# Figures

---

## Table of Contents 

* [1. Introduction](#introduction)
    * [1.1. Project Description](#project-description)
    * [1.2. Packages](#packges)


* [2. Data Loading](#data-loading)

    
* [3. Main Figures](#main)
    * [3.1 - Figure 1. Schema and model selection, training and feature importance in the final RF model](#figure1)
    * [3.2 - Figure 2. TRIFID learning curve and feature importance](#figure2)
    * [3.3 - Figure 3. Length and conservation scores versus TRIFID scores](#figure3)
    * [3.4 - Figure 4. Normalized TRIFID scores and for alternative and principal isoforms](#figure4)
    * [3.5 - Figure 5. A schematic illustration of the two functionally important ERCC6 isoforms](#figure5)
    * [3.6 - Figure 6. Model predictions for four fibroblast growth factor receptor 1 (FGFR1) isoforms ](#figure6)
    * [3.7 - Figure 7. A comparison between TRIFID and PULSE and exporting TRIFID to other species](#figure7)
    * [3.8 - Figure 8.TRIFID scores and genomic variation for principal and alternative exons](#figure8)

    
* [4. Supplementary figures](#supplementary)
    * [4.1 - Figure S1. The relationship between the 45 predictive features in TRIFID](#figures1)
    * [4.2 - Figure S2: Length and conservation scores versus TRIFID scores](#figures2)
    * [4.3 - Figure S3: Distribution of TRIFID scores for GENCODE v27 isoforms](#figures3)
    * [4.4 - Figure S4: Distribution of TRIFID scores for principal and alternative isoforms](#figures4)
    * [4.5 - Figure S5: Distribution of TRIFID scores for GENCODE v27 transcript types](#figures5)
    * [4.6 - Figure S6: A scatter plot of Raw TRIFID score against Normalised TRIFID score](#figures6)
    * [4.7 - Figure S7: SHAP local predictions for ERCC6 isoforms](#figures7)
    * [4.8 - Figure S8: SHAP local predictions for FGFR1 isoforms](#figures8)
    * [4.9 - Figure S9: Further examples of discrepancies between TRIFID and PULSE scores](#figures9)
    * [4.10 - Figure S10: Human - generic comparison](#figures10)
    * [4.11 - Extra figures](#figures10)
    

* [5. References](#references)

* [6. Project contribution](#project-contribution)

---

# 1. Introduction <a class="anchor" id="introduction"></a>

## 1.1. Project Description <a class="anchor" id="project-description"></a>

Alternative Splicing (AS) of messenger RNA can generate a wide variety of mature RNA transcripts and this expression is confirmed by experimental transcript evidence. In theory these transcripts could generate protein isoforms with diverse cellular functions. However, while peptide evidence strongly supports a main protein isoform for the vast majority of coding genes [(1)](https://pubs.acs.org/doi/full/10.1021/pr501286b), it is not clear what proportion of these AS isoforms form stable functional proteins. In fact reliable proteomics experiments have found little evidence of alternative spliced proteins [(2)](https://www.sciencedirect.com/science/article/pii/S0968000416301189), so the number of stably folded/functional proteins produced by AS remains a mystery.

We have developed a computational method (`TRIFID`) for the classification of splice isoform functional importance. This algorithm was trained on reliable peptide evidence from proteomics analyses and classifies biologically important splice isoforms with high confidence. The algorithm ranks the most significant biological splice isoforms and we show that the highest scoring alternative exons are actually under selection pressure, unlike the vast majority of alternative exons. TRIFID can predict functional isoforms for any well-annotated eukaryotic species. The method will generate valuable insights into the cellular importance of alternative splicing.

## 1.2. Packages <a class="anchor" id="packages"></a>

In [ ]:
import os, sys, pickle, warnings

import pandas as pd
import numpy as np
import altair as alt 
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef, make_scorer
import scipy.stats as stats

sys.path.append('../')
from trifid.data.loaders import Fasta
from trifid.models.select import Classifier
from trifid.models.interpret import TreeInterpretation
from trifid.utils.utils import *
from trifid.visualization.figures import * 

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
warnings.filterwarnings("ignore")
alt.data_transformers.disable_max_rows()
sns.set_style("whitegrid")

%matplotlib inline
%load_ext watermark
%watermark -a 'Fernando Pozo' -u -n -t -z -g -p pandas,numpy,altair,matplotlib,sklearn,yellowbrick

%load_ext autoreload
%autoreload 2

!pwd

---

# 2. Data Loading <a class="anchor" id="data-loading"></a>

In [ ]:
# initial parameters
TRIFID_DIR = '/local/fpozoc/Projects/trifid'
CONFIG_PATH = os.path.join(TRIFID_DIR, 'config/config.yaml')
FEATURES_PATH = os.path.join(TRIFID_DIR, 'config/features.yaml')
SEED = 123

# features
config = parse_yaml(CONFIG_PATH)
df_features = pd.DataFrame(parse_yaml(FEATURES_PATH))
features = df_features[~df_features['category'].str.contains('Identifier')]['feature'].values

# feature dataset
df_g27 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')
df_g27_features = df_g27[features]
df_g35 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g35', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')
df_rs109 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'rs109', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')

# training set and predictions
df_training_set = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'model', 'training_set_initial.g27.tsv.gz'), sep='\t')
df_training_set.loc[df_training_set['state'].str.contains('F'), 'label'] = 1
df_training_set.loc[df_training_set['state'].str.contains('U'), 'label'] = 0
df_training_set = df_training_set.loc[~df_training_set['label'].isnull()]
df_training_set = df_training_set.loc[df_training_set['added'].str.contains('v1|r|v3')].drop(['added', 'state', 'comment'], axis=1).reset_index(drop=True)

df_g27_sequences = Fasta(os.path.join(TRIFID_DIR, 'data', 'source', 'GRCh38', 'g27', 'appris_data.transl.fa.gz'), db='g').load
df_g27_sequences['transcript_id'] = df_g27_sequences['id'].str.split('|').str[1]
df_g27_sequences = df_g27_sequences[['transcript_id', 'sequence']]

df_g27_predictions = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27', 'trifid_predictions.tsv.gz'), sep='\t', compression='gzip')
df_g35_predictions = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g35', 'trifid_predictions.tsv.gz'), sep='\t', compression='gzip')
df_rs109_predictions = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'rs109', 'trifid_predictions.tsv.gz'), sep='\t', compression='gzip')
df_gm25_predictions = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCm38', 'g25', 'trifid_predictions.tsv.gz'), sep='\t', compression='gzip')

# model
my_model = pickle.load(open(os.path.join(TRIFID_DIR, 'models', 'selected_model.pkl'), 'rb'))
model = Classifier(
    model = my_model,
    df = df_training_set, 
    features_col=df_training_set[features].columns,
    target_col='label',
    random_state=SEED)

# interpretation
interpretation = TreeInterpretation(
    model = my_model,
    df = df_training_set, 
    features_col=df_training_set[features].columns,
    target_col='label',
    random_state=SEED,
    test_size=0.25)

---

In [1]:
class PDF(object):
    def __init__(self, pdf, size=(200,200)):
        self.pdf = pdf
        self.size = size
    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)
    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

# 3. Main figures <a class="anchor" id="main"></a>

## 3.1 - Figure 1. Schema and model selection, training and feature importance in the final RF model <a class="anchor" id="figure1"></a>

In [2]:
PDF('../reports/figures/fig1.pdf',size=(600,600))

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(4, 4))
plot_prcurve(
    model=model.model,
    X=model.train_features.values, 
    y=model.train_target.values, 
    n_splits=10,
    seed=123,
    ax=ax1)
fig.show()
# fig.savefig('../reports/figures/fig1C.png', bbox_inches='tight' ,dpi=1200)

## 3.2 - Figure 2. TRIFID learning curve and feature importance <a class="anchor" id="figure2"></a>

In [3]:
PDF('../reports/figures/fig2.pdf',size=(800,600))

In [ ]:
# shap dataframe processing
df_shap = interpretation.shap
df_shap['shap_percentage'] = (df_shap['shap']/df_shap['shap'].sum())*100
df_shap = pd.merge(df_features[df_features['category'] != 'Identifier'], df_shap, on='feature', how='left')
df_shap['group_percentage'] = df_shap.groupby('category')['shap_percentage'].transform(lambda x: x.sum()).round(2)
df_shap['category'] = df_shap.apply(lambda row: str(row['category']) + ' (' + str(row['group_percentage']) + ' %)', axis=1)

In [ ]:
df_shap.drop('group_percentage', axis=1).to_csv(os.path.join(TRIFID_DIR, 'reports', 'tables', 'trifid_features.g27.tsv.gz'), index=None, sep='\t', compression='gzip')

In [ ]:
plot_feature_importances(df_shap, xcol='shap_percentage', ycol='name', facetcol='category', method='SHAP', ntop=18)

In [ ]:
plt_vc_mcc = plot_learning_curve(model = my_model, seed = SEED, train_size = np.linspace(0.1, 1, 10),
                    scoring_function = make_scorer(matthews_corrcoef),
                    features = pd.concat([model.train_features, model.test_features]).reset_index(drop=True),
                    target = pd.concat([model.train_target, model.test_target]).reset_index(drop=True),
                    ylabel = 'Matthews correlation coefficient', 
                    ylim = [0.825, 0.96]
                   )
# plt_vc_ap.savefig('../reports/figures/fig2A.png', bbox_inches='tight' ,dpi=480)

In [ ]:
plt_vc_ap = plot_learning_curve(model = my_model, seed = SEED, train_size = np.linspace(0.1, 1, 10),
                    scoring_function = 'average_precision',
                    features = pd.concat([model.train_features, model.test_features]).reset_index(drop=True),
                    target = pd.concat([model.train_target, model.test_target]).reset_index(drop=True),
                    ylabel = 'Average precision',
                    ylim = [0.95, 1]
                   )
# plt_vc_ap.savefig('../reports/figures/fig2B.png', bbox_inches='tight' ,dpi=480)

## 3.3 - Figure 3. Length and conservation scores versus TRIFID scores <a class="anchor" id="figure3"></a>

In [4]:
PDF('../reports/figures/fig3.pdf',size=(700,400))

In [ ]:
df_bilateria = pd.read_excel('../reports/data/F-M_F-M.xlsx')
df_bilateria = df_bilateria.rename(columns={'HGNC (G24)': 'gene_name', 'Ensembl': 'gene_id'})
df_bilateria = df_bilateria.drop('gene_name', axis=1)

df_g27_bilateria = pd.merge(df_bilateria, df_g27, how='left', on='gene_id')

data1_bil = create_categories(df_g27_bilateria, df_g27_predictions, feature='length', cats=[1000, 2000, 2500])
data1_bil['Gene set'] = f'Bilateria ({data1_bil.index.nunique()} genes)'
data2_bil = create_categories(df_g27_bilateria, df_g27_predictions, feature='corsair', cats=[1.5, 4, 10])
data2_bil['Gene set'] = f'Bilateria ({data2_bil.index.nunique()} genes)'
data3_bil = create_categories(df_g27_bilateria, df_g27_predictions, feature='corsair_alt', cats=[0.25, 0.5, 0.75, 1])
data3_bil['Gene set'] = f'Bilateria ({data3_bil.index.nunique()} genes)'

plt.rc('axes', titlesize=16, labelsize=18)   
plt.rc('ytick', labelsize=18)    # fontsize of the tick labels

fig, axes = plt.subplots(3, 1, sharex=True, sharey=False, figsize=(14, 8))
sns.despine(trim=False, left=False)

palette = sns.color_palette("coolwarm", 2)
axes[0] = sns.boxplot(x="trifid_score", y="Length", data=data1_bil, orient='w', width=.7, fliersize=1, color="#d9d9d9", ax=axes[0])
axes[1] = sns.boxplot(x="trifid_score", y="Corsair", data=data2_bil, orient='w', width=.7, fliersize=1, color="#a4c2f4", ax=axes[1])
axes[2] = sns.boxplot(x="trifid_score", y="Corsair_alt", data=data3_bil, orient='w', width=.7, fliersize=1, color="#4a86e8", ax=axes[2])

for ax in axes:
    ax.set_xlabel("")
    ax.xaxis.grid(False)
    ax.yaxis.grid(False)
    
axes[0].set_ylabel('Length', labelpad=26, fontsize=20)
axes[0].margins(0)
axes[1].set_ylabel('CORSAIR', labelpad=40, fontsize=20)
axes[2].set_ylabel('Alt-CORSAIR', labelpad=30, fontsize=20)
axes[2].set_xlabel("TRIFID score", fontsize=20)
plt.tick_params(labelsize=18)
plt.xlim([0,1])
plt.tight_layout()
# plt.savefig('../reports/figures/fig3.png', bbox_inches='tight', dpi=350)
# plt.savefig('../reports/figures/fig3.pdf', bbox_inches='tight', dpi=350)

## 3.4 - Figure 4. Normalized TRIFID scores and for alternative and principal isoforms <a class="anchor" id="figure4"></a>

In [6]:
PDF('../reports/figures/fig4.pdf',size=(700,400))

## 3.5 - Figure 5. A schematic illustration of the two functionally important ERCC6 isoforms <a class="anchor" id="figure5"></a>

In [7]:
PDF('../reports/figures/fig5.pdf',size=(1000,600))

## 3.6 - Figure 6. Model predictions for four fibroblast growth factor receptor 1 (FGFR1) isoforms <a class="anchor" id="figure6"></a>

In [8]:
PDF('../reports/figures/fig6.pdf',size=(1000,600))

## 3.7 - Figure 7. A comparison between TRIFID and PULSE and exporting TRIFID to other species <a class="anchor" id="figure7"></a>

In [9]:
PDF('../reports/figures/fig7.pdf',size=(1000,600))

In [ ]:
df_g27_predictions = pd.merge(df_g27_predictions, df_g27_sequences, how='left', on='transcript_id')

In [ ]:
# pulse dataframe processing
df_pulse = pd.read_excel('../references/pulse/predictions/1-s2.0-S2211124715006439-mmc3.xlsx', skiprows=18)
df_pulse = df_pulse.rename(columns={'Sequence': 'sequence', 'Score': 'PULSE'})
df_g27_pulse = pd.merge(df_pulse[['sequence', 'PULSE', 'ID', 'CanonicalAnchor', 'AlternativeIsoformIDS', 'CanonicalSequence']], 
                        df_g27_predictions[['gene_id','gene_name', 'transcript_id', 'appris', 'flags', 'sequence', 'trifid_score', 'norm_trifid_score']], 
                        how='left', on='sequence')

# shap top 20 features
features_top_20 = list(df_shap.sort_values(by='shap', ascending=False).head(20)['feature'].values)

df_g27_pulse = pd.merge(df_g27_pulse, 
                        df_g27[['transcript_id'] + features_top_20],
                        on='transcript_id', how='left'
                       )
df_g27_pulse = df_g27_pulse.loc[~df_g27_pulse['transcript_id'].isnull()].reset_index(drop=True)
df_g27_pulse = df_g27_pulse.loc[~df_g27_pulse['appris'].str.contains('PRINCIPAL')]
df_g27_pulse = df_g27_pulse.loc[~df_g27_pulse['flags'].str.contains('nonsense_mediated_decay')]
df_g27_pulse = df_g27_pulse.drop_duplicates('sequence').reset_index(drop=True)

In [ ]:
r_pearson, p_pearson = stats.pearsonr(df_g27_pulse['norm_trifid_score'], df_g27_pulse['PULSE'])
r_spearman, p_spearman = stats.spearmanr(df_g27_pulse['norm_trifid_score'], df_g27_pulse['PULSE'])
legend_label = f'Pearson $\\rho$ = {r_pearson.round(3)}; Spearman $\\rho$ = {r_spearman.round(3)}'
print(legend_label)

scp = sns.jointplot(data=df_g27_pulse, 
              x="norm_trifid_score", y="PULSE", 
              kind="reg", color='#3CAEA3', 
              xlim=(0,1), ylim=(0,1), 
              height=8, ratio=6,
              space=0.3,
              marginal_kws=dict(bins=20, rug=True),
              joint_kws = {'scatter_kws':dict(alpha=0.7, s=20)}
                   )
# scp.ax_joint.annotate(legend_label, xy=(0.3, 0.05), xycoords='axes fraction', fontsize=16)

plt.xlabel('Normalized TRIFID score', fontsize=34)
plt.xticks(fontsize=28)

plt.ylabel('PULSE predicted score', fontsize=34)
plt.yticks(fontsize=28)
plt.tight_layout()
# plt.savefig('../reports/figures/fig7A.1.pdf', bbox_inches='tight', dpi=350)
# plt.savefig('../reports/figures/fig7A.1.png', bbox_inches='tight', dpi=350)

## 3.8 - Figure 8.TRIFID scores and genomic variation for principal and alternative exons <a class="anchor" id="figure8"></a>

In [10]:
PDF('../reports/figures/fig8.pdf',size=(600,500))

# 3. Supplementary figures <a class="anchor" id="supplementary"></a>

## 4.1 - Figure S1. The relationship between the 45 predictive features in TRIFID <a class="anchor" id="figures1"></a>

In [11]:
PDF('../reports/figures/figS1.pdf',size=(1000,1000))

In [ ]:
df = df_g27[df_g27.columns[8:]]

features_names = df_shap.set_index('feature')['name'].to_dict()
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
    df = df.to_frame(index=False)

# remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
df = df.reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers

corr_cols = features
corr_data = np.corrcoef(df[corr_cols].values, rowvar=False)
corr_data = pd.DataFrame(corr_data, columns=[corr_cols], index=[corr_cols])
corr_data = corr_data.rename(index=features_names, columns=features_names).round(2)
corr_data.index.name = str('column')

sns.set(font_scale=1.4)
plt.figure(figsize=(30, 24))
heatmap = sns.heatmap(corr_data, vmin=-1, vmax=1, annot=True,  cmap='RdBu', linewidths=1, fmt='.2g', linecolor='white',  annot_kws={"size":12})
plt.xlabel('')
plt.ylabel('')
# plt.savefig('../reports/figures/figS1.png', bbox_inches='tight')
# plt.savefig('../reports/figures/figS1.pdf', dpi=350, bbox_inches='tight')

## 4.2 - Figure S2: Length and conservation scores versus TRIFID scores <a class="anchor" id="figures2"></a>

In [12]:
PDF('../reports/figures/figS2.pdf',size=(700,400))

In [ ]:
sns.set_style("whitegrid")
df_g27_nopnc = df_g27.loc[~df_g27['gene_id'].str.contains('$|^'.join(list(df_notcoding['gene_id'].values)))]
df_notcoding = pd.read_csv('../reports/data/PNC_genes_G27.tsv', names=['gene_id'])
df_g27_notcoding = pd.merge(df_notcoding, df_g27, how='left', on='gene_id')

data1_g27 = create_categories(df_g27_nopnc, df_g27_predictions, feature='length', cats=[1000, 2000, 2500])
data1_pnc = create_categories(df_g27_notcoding, df_g27_predictions, feature='length', cats=[1000, 2000, 2500])
data1_g27['Gene set'] = f'Non PNC ({data1_g27.index.nunique()} genes)'
data1_pnc['Gene set'] = f'PNC ({data1_pnc.index.nunique()} genes)'

data2_g27 = create_categories(df_g27_nopnc, df_g27_predictions, feature='corsair', cats=[1.5, 4, 10])
data2_pnc = create_categories(df_g27_notcoding, df_g27_predictions, feature='corsair', cats=[1.5, 4, 10])
data2_g27['Gene set'] = f'Non PNC ({data2_g27.index.nunique()} genes)'
data2_pnc['Gene set'] = f'PNC ({data2_pnc.index.nunique()} genes)'

data3_g27 = create_categories(df_g27_nopnc, df_g27_predictions, feature='corsair_alt', cats=[0.25, 0.5, 0.75, 1])
data3_pnc = create_categories(df_g27_notcoding, df_g27_predictions, feature='corsair_alt', cats=[0.25, 0.5, 0.75, 1])
data3_g27['Gene set'] = f'Non PNC ({data3_g27.index.nunique()} genes)'
data3_pnc['Gene set'] = f'PNC ({data3_pnc.index.nunique()} genes)'

plt.rc('axes', titlesize=16, labelsize=18)   
plt.rc('ytick', labelsize=18)    # fontsize of the tick labels

fig, axes = plt.subplots(3, 1, sharex=True, sharey=False, figsize=(16, 10))

sns.despine(trim=False, left=False)

palette = sns.color_palette('coolwarm', 3)[1:]
axes[0] = sns.boxplot(x="trifid_score", y="Length", data=pd.concat([data1_g27, data1_pnc]), orient='w', width=.7, fliersize=1, color="#a4c2f4", ax=axes[0], hue="Gene set", palette=palette)
axes[1] = sns.boxplot(x="trifid_score", y="Corsair", data=pd.concat([data2_g27, data2_pnc]), orient='w', width=.7, fliersize=1, color="#a4c2f4", ax=axes[1], hue="Gene set", palette=palette)
axes[2] = sns.boxplot(x="trifid_score", y="Corsair_alt", data=pd.concat([data3_g27, data3_pnc]), orient='w', width=.7, fliersize=1, color="#a4c2f4", ax=axes[2], hue="Gene set", palette=palette)

for ax in axes:
    ax.set_xlabel("")
    ax.xaxis.grid(False)
    ax.yaxis.grid(False)
    
axes[0].set_ylabel('Length', labelpad=26, fontsize=20)
axes[0].margins(0)
axes[0].get_legend().remove()
axes[1].set_ylabel('CORSAIR', labelpad=40, fontsize=20)
axes[1].get_legend().remove()
axes[2].set_ylabel('Alt-CORSAIR', labelpad=30, fontsize=20)
axes[2].set_xlabel("TRIFID score", fontsize=20)
axes[2].get_legend().remove()

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='left', prop={'size': 18}, frameon=True, bbox_to_anchor=(0,0,0.97,1.09))

plt.tick_params(labelsize=18)
plt.xlim([0,1])
plt.tight_layout()
# plt.savefig('../reports/figures/figS2.png', dpi=350, bbox_inches='tight')
# plt.savefig('../reports/figures/figS2.pdf', dpi=350, bbox_inches='tight')

## 4.3 - Figure S3: Distribution of TRIFID scores for GENCODE v27 isoforms <a class="anchor" id="figures3"></a>

In [13]:
PDF('../reports/figures/figS3.pdf',size=(700,300))

In [ ]:
xcol='trifid_score'
palette='set2'
opacity=0.8
chart = alt.Chart(df_g27_predictions[['trifid_score']]).mark_bar().encode(
    x=alt.X(xcol,
            bin=alt.Bin(extent=[0, 1],
                        step=0.025),
            axis=alt.Axis(title='TRIFID Score', 
                         titleFont='Arial',
                         titleFontWeight='normal',
                         titleFontSize=22,
                         labelFont='Arial', 
                         labelFontSize=16)),
    y=alt.Y('count()', axis=alt.Axis(title='', titleFont='Arial', titleFontWeight='normal', titleFontSize=18,
                                    labelFont='Arial', labelFontSize=16)),
    opacity=alt.value(opacity),
)
config_altair(chart, height=200, width=600)

## 4.4 - Figure S4: Distribution of TRIFID scores for principal and alternative isoforms <a class="anchor" id="figures4"></a>

In [14]:
PDF('../reports/figures/figS4.pdf',size=(700,600))

In [ ]:
plot_appris_histogram(cat_appris_order(df_g27_predictions))

## 4.5 - Figure S5: Distribution of TRIFID scores for GENCODE v27 transcript types <a class="anchor" id="figures5"></a>

In [15]:
PDF('../reports/figures/figS5.pdf',size=(700,500))

In [ ]:
plot_transcript_types_histogram(cat_transcript_type(df_g27_predictions))

## 4.6 - Figure S6: A scatter plot of Raw TRIFID score against Normalised TRIFID score <a class="anchor" id="figures6"></a>

In [16]:
PDF('../reports/figures/figS6.pdf',size=(700,500))

In [ ]:
r_pearson, p_pearson = stats.pearsonr(df_g27_predictions['trifid_score'], df_g27_predictions['norm_trifid_score'])
r_spearman, p_spearman = stats.spearmanr(df_g27_predictions['trifid_score'], df_g27_predictions['norm_trifid_score'])
legend_label = f'Pearson $\\rho$ = {r_pearson.round(3)}; Spearman $\\rho$ = {r_spearman.round(3)}'
print(legend_label)

scp = sns.jointplot(data=df_g27_predictions, 
              x="trifid_score", y="norm_trifid_score", 
              kind="reg", color='#ED553B', 
              xlim=(0,1), ylim=(0,1), 
              height=8, ratio=6,
              joint_kws = {'scatter_kws':dict(alpha=0.25, s=10)}
                   )
# scp.ax_joint.annotate(legend_label, xy=(0.3, 0.05), xycoords='axes fraction', fontsize=16)

plt.xlabel('TRIFID score', fontsize=20)
plt.xticks(fontsize=18)

plt.ylabel('Normalized TRIFID score', fontsize=20)
plt.yticks(fontsize=18)
plt.tight_layout()
# plt.savefig('../reports/figures/figS6.pdf', dpi=350, bbox_inches='tight')
# plt.savefig('../reports/figures/figS6.png', dpi=350, bbox_inches='tight')

## 4.7 - Figure S7: SHAP local predictions for ERCC6 isoforms <a class="anchor" id="figures7"></a>

In [17]:
PDF('../reports/figures/figS7.pdf',size=(800,400))

In [ ]:
df_local_shap = pd.merge(df_g27_predictions[['transcript_id', 'trifid_score']], df_g27, how='left', on='transcript_id').round(2)

In [ ]:
ercc6_transcripts = df_local_shap[df_local_shap.gene_name == 'ERCC6']['transcript_id'].values
ercc6_transcripts = ['ENST00000355832', 'ENST00000447839']

In [ ]:
for n, tid in enumerate(ercc6_transcripts):
    explain_prediction(df_local_shap, model, features, tid)
    # plt.savefig(f'../reports/figures/figS7.{n+1}.png', bbox_inches='tight', dpi=350)

## 4.8 - Figure S8: SHAP local predictions for FGFR1 isoforms <a class="anchor" id="figures8"></a>

In [18]:
PDF('../reports/figures/figS8.pdf',size=(700,600))

In [ ]:
fgfr1_transcripts = df_local_shap[df_local_shap.gene_name == 'FGFR1']['transcript_id'].values
fgfr1_transcripts = ['ENST00000447712', 'ENST00000397103', 'ENST00000619564', 'ENST00000356207']

In [ ]:
for n, tid in enumerate(fgfr1_transcripts):
    explain_prediction(df_local_shap, model, features, tid)
    # plt.savefig(f'../reports/figures/figS8.{n+1}.png', bbox_inches='tight', dpi=350)

In [ ]:
explain_prediction(df_local_shap, model, features, 'ENST00000356207')
# plt.savefig(f'../img/local_shap_fgfr1.png', bbox_inches='tight', dpi=350)

## 4.9 - Figure S9: Further examples of discrepancies between TRIFID and PULSE scores <a class="anchor" id="figures9"></a>

In [19]:
PDF('../reports/figures/figS9.pdf',size=(800,500))

## 4.10 - Figure S10: Human - generic comparison <a class="anchor" id="figures10"></a>

In [20]:
PDF('../reports/figures/figS10.pdf',size=(800,400))

In [ ]:
generic_features = df_features[df_features['support'] == 'cross-species'].loc[df_features['category'] != 'Identifier'].feature.values

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4))
fig.subplots_adjust(hspace = 2)
plot_prcurve(
    model=model.model,
    X=pd.concat([model.train_features, model.test_features]).reset_index(drop=True).values, 
    y=pd.concat([model.train_target, model.test_target]).reset_index(drop=True).values, 
    n_splits=5,
    seed=120,
    title='',
    ax=ax1)

plot_prcurve(
    model=model.model,
    X=pd.concat([model.train_features[generic_features], model.test_features[generic_features]]).reset_index(drop=True).values, 
    y=pd.concat([model.train_target, model.test_target]).reset_index(drop=True).values, 
    n_splits=5,
    seed=120,
    title='',
    ax=ax2)

# plt.savefig('../reports/figures/figS10.png', dpi=350, bbox_inches='tight')
# plt.savefig('../reports/figures/figS10.pdf', dpi=350, bbox_inches='tight')

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(4, 4))
plot_prcurve(
    model=model.model,
    X=pd.concat([model.train_features, model.test_features]).reset_index(drop=True).values, 
    y=pd.concat([model.train_target, model.test_target]).reset_index(drop=True).values, 
    n_splits=5,
    seed=120,
    title='',
    ax=ax1)
# fig.savefig('../reports/figures/figS10A.png', bbox_inches='tight' ,dpi=360)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(4, 4))
plot_prcurve(
    model=model.model,
    X=pd.concat([model.train_features[features_generic], model.test_features[features_generic]]).reset_index(drop=True).values, 
    y=pd.concat([model.train_target, model.test_target]).reset_index(drop=True).values, 
    n_splits=5,
    seed=120,
    title='',
    ax=ax1)
# fig.savefig('../reports/figures/figS10B.png', bbox_inches='tight' ,dpi=360)

# 5. References <a class="anchor" id="references"></a>

- pandas 1.0.4.
- numpy 1.18.5.
- altair 4.1.0.
- matplotlib 3.2.1.
- sklearn 0.23.1.
- yellowbrick 1.1.

-----

# 6. Project contribution <a class="anchor" id="project-contribution"></a>

**Author information**:

Fernando Pozo

- [ORCID iD (0000-0001-7688-6045)](https://orcid.org/0000-0001-7688-6045)
- [GitLab (@fpozoc)](https://gitlab.com/fpozoc)
- [Twitter (@fpozoca)](https://twitter.com/fpozoca)